In [1]:
from collections import defaultdict
import numpy as np

# Leer el archivo FASTA
with open('protein.fasta', 'r') as f:
    secuencia_proteina = f.readlines()[1].strip()  # Saltar la primera línea

# Inicializar las matrices F2B y B2F como diccionarios de diccionarios
F2B = defaultdict(lambda: defaultdict(int))
B2F = defaultdict(lambda: defaultdict(int))

# Calcular las matrices F2B y B2F
for i in range(len(secuencia_proteina) - 1):
    F2B[secuencia_proteina[i]][secuencia_proteina[i+1]] += 1
    B2F[secuencia_proteina[i+1]][secuencia_proteina[i]] += 1

# Normalizar las matrices F2B y B2F
for dominio in F2B:
    total = sum(F2B[dominio].values())
    for siguiente_dominio in F2B[dominio]:
        F2B[dominio][siguiente_dominio] /= total

for dominio in B2F:
    total = sum(B2F[dominio].values())
    for anterior_dominio in B2F[dominio]:
        B2F[dominio][anterior_dominio] /= total

# Imprimir las matrices F2B y B2F
print("Matriz F2B:")
for dominio in sorted(F2B.keys()):
    for siguiente_dominio in sorted(F2B[dominio].keys()):
        print(f"{dominio}{siguiente_dominio}: {F2B[dominio][siguiente_dominio]}")

print("\nMatriz B2F:")
for dominio in sorted(B2F.keys()):
    for anterior_dominio in sorted(B2F[dominio].keys()):
        print(f"{anterior_dominio}{dominio}: {B2F[dominio][anterior_dominio]}")


Matriz F2B:
AA: 0.3897338403041825
AC: 0.09378960709759189
AG: 0.17490494296577946
AT: 0.34157160963244615
CA: 0.4517766497461929
CC: 0.12690355329949238
CG: 0.0583756345177665
CT: 0.3629441624365482
GA: 0.3958664546899841
GC: 0.12241653418124006
GG: 0.2066772655007949
GT: 0.27503974562798095
TA: 0.41582622187742435
TC: 0.09231962761830877
TG: 0.1551590380139643
TT: 0.33669511249030254

Matriz B2F:
AA: 0.3897338403041825
CA: 0.11280101394169835
GA: 0.15779467680608364
TA: 0.3396704689480355
AC: 0.3756345177664975
CC: 0.12690355329949238
GC: 0.19543147208121828
TC: 0.3020304568527919
AG: 0.43879173290937995
CG: 0.03656597774244833
GG: 0.2066772655007949
TG: 0.3179650238473768
AT: 0.4181536074476338
CT: 0.11093871217998448
GT: 0.13421256788207914
TT: 0.33669511249030254


In [2]:
class AutomataCelular:
    def __init__(self, m, g, dominio_ancestral, F2B, B2F):
        self.m = m  # Número de dominios en la proteína con más dominios
        self.g = g  # Número de clases de dominios en el conjunto de datos de entrenamiento        
        self.CA = np.array(['∅' for _ in range(2*m+1)])  # CA de 1D
        self.CA[m] = dominio_ancestral  # Establecer la celda del medio como el dominio ancestral
        self.F2B = F2B  # Matriz de probabilidades previas de evolución de adelante hacia atrás
        self.B2F = B2F  # Matriz de probabilidades previas de evolución de atrás hacia adelante

    def seleccion_ruleta(self, probabilidades):
        max_val = sum(probabilidades.values())
        seleccion = np.random.uniform(0, max_val)
        actual = 0
        for clave, valor in probabilidades.items():
            actual += valor
            if actual > seleccion:
                return clave

    def actualizar_estado(self, t):
        for j in range(1, 2*self.m):  # Saltar la primera y última celdas
            if self.CA[j] != '∅':
                # Regla A: Herencia
                self.CA[j] = self.CA[j]
            elif self.CA[j-1] != '∅' and self.CA[j+1] == '∅':
                # Regla B: Evolución de adelante hacia atrás
                self.CA[j] = self.seleccion_ruleta(self.F2B[self.CA[j-1]])
            elif self.CA[j-1] == '∅' and self.CA[j+1] != '∅':
                # Regla C: Evolución de atrás hacia adelante
                self.CA[j] = self.seleccion_ruleta(self.B2F[self.CA[j+1]])                
            else:
                # Regla D: Volver a inanimidad
                self.CA[j] = '∅'

    def ejecutar(self):
        for t in range(self.m):
            print(self.CA)
            self.actualizar_estado(t)


In [3]:
# Create an instance of the CellularAutomaton class
CA = AutomataCelular(m=8, g=4, dominio_ancestral='A', F2B=F2B, B2F=B2F)
CA.ejecutar()
print(CA.CA)

['∅' '∅' '∅' '∅' '∅' '∅' '∅' '∅' 'A' '∅' '∅' '∅' '∅' '∅' '∅' '∅' '∅']
['∅' '∅' '∅' '∅' '∅' '∅' '∅' 'A' 'A' 'C' 'A' 'G' 'A' 'T' 'G' 'A' '∅']
['∅' '∅' '∅' '∅' '∅' '∅' 'G' 'A' 'A' 'C' 'A' 'G' 'A' 'T' 'G' 'A' '∅']
['∅' '∅' '∅' '∅' '∅' 'G' 'G' 'A' 'A' 'C' 'A' 'G' 'A' 'T' 'G' 'A' '∅']
['∅' '∅' '∅' '∅' 'A' 'G' 'G' 'A' 'A' 'C' 'A' 'G' 'A' 'T' 'G' 'A' '∅']
['∅' '∅' '∅' 'G' 'A' 'G' 'G' 'A' 'A' 'C' 'A' 'G' 'A' 'T' 'G' 'A' '∅']
['∅' '∅' 'A' 'G' 'A' 'G' 'G' 'A' 'A' 'C' 'A' 'G' 'A' 'T' 'G' 'A' '∅']
['∅' 'T' 'A' 'G' 'A' 'G' 'G' 'A' 'A' 'C' 'A' 'G' 'A' 'T' 'G' 'A' '∅']
['∅' 'T' 'A' 'G' 'A' 'G' 'G' 'A' 'A' 'C' 'A' 'G' 'A' 'T' 'G' 'A' '∅']
